In [1]:
import fastapi
import uvicorn
from utils.Database import Database
from utils.Copernicus import AdvancedCopernicus
from utils.OpenMeteoWeather import OpenMeteoWeather
from utils.PlanetPositions import PlanetPositions
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

In [3]:
print(datetime.datetime.now().isoformat())

2025-02-22T16:13:17.934723


In [4]:
# ------------ Initialize the global Variables ------------

# Define Absolute maximum and minimum values for date and location
ABSOLUTE_END_DATE:str = datetime.datetime.now().isoformat().split("T")[0]

ABSOLUTE_MINIMUM_LONGITUDE:float = 9.041532516479492
ABSOLUTE_MAXIMUM_LONGITUDE:float = 30.208656311035156
ABSOLUTE_MINIMUM_LATIDUDE:float = 53.00829315185547
ABSOLUTE_MAXIMUM_LATIDUDE:float = 65.89141845703125




START_DATE:str = "2025-01-01"
END_DATE:str = "2025-02-1"



MINIMUM_LONGITUDE:float = 9.59
MAXIMUM_LONGITUDE:float = 12.037
MINIMUM_LATIDUDE:float = 55.486
MAXIMUM_LATIDUDE:float = 55.616


# For Testing
MINIMUM_LONGITUDE=10.038345850696412
MAXIMUM_LONGITUDE=10.365962458698567
MINIMUM_LATIDUDE=54.27381478077755
MAXIMUM_LATIDUDE=54.52976525577923

OUTPUT_FILENAME:str = "output.nc"


In [5]:
# ------------ Initialize the classes ------------
# db = Database(
#     db_url='localhost',
#     db_name='deep-learning',
#     collection_name='test'
# )

    

# db.close_connection()
AdvancedCopernicus = AdvancedCopernicus()
#OpenMeteoWeather = OpenMeteoWeather()
#PlanetPositions = PlanetPositions()

In [6]:
# ------------ Get data from AdvancedCopernicus ------------
copernicus_data = AdvancedCopernicus.get_subset(
                dataset_id="cmems_mod_bal_phy_anfc_PT1H-i",
                dataset_version="202411",
                variables=["bottomT", "mlotst", "siconc", "sithick", "sla", "so", "sob", "thetao", "uo", "vo", "wo"], 
                minimum_longitude=MINIMUM_LONGITUDE,
                maximum_longitude=MAXIMUM_LONGITUDE,
                minimum_latitude=MINIMUM_LATIDUDE,
                maximum_latitude=MAXIMUM_LATIDUDE,
                start_datetime=START_DATE,
                end_datetime=END_DATE,
                minimum_depth=0.5016462206840515,
                maximum_depth=0.5016462206840515,
                coordinates_selection_method="strict-inside",
                disable_progress_bar=False,
                output_filename=OUTPUT_FILENAME
                )
            



INFO - 2025-02-22T15:17:00Z - Selected dataset version: "202411"
INFO - 2025-02-22T15:17:00Z - Selected dataset part: "default"
INFO - 2025-02-22T15:17:08Z - Starting download. Please wait...


  0%|          | 0/44 [00:00<?, ?it/s]

INFO - 2025-02-22T15:17:14Z - Successfully downloaded to output.nc


In [7]:
# plot xarray dataset in map
print(copernicus_data)


<xarray.Dataset> Size: 6MB
Dimensions:    (depth: 1, latitude: 16, longitude: 12, time: 745)
Coordinates:
  * depth      (depth) float32 4B 0.5016
  * latitude   (latitude) float32 64B 54.27 54.29 54.31 ... 54.49 54.51 54.52
  * longitude  (longitude) float32 48B 10.04 10.07 10.1 ... 10.29 10.32 10.35
  * time       (time) datetime64[ns] 6kB 2025-01-01 ... 2025-02-01
Data variables:
    bottomT    (time, latitude, longitude) float32 572kB ...
    mlotst     (time, latitude, longitude) float32 572kB ...
    siconc     (time, latitude, longitude) float32 572kB ...
    sithick    (time, latitude, longitude) float32 572kB ...
    sla        (time, latitude, longitude) float32 572kB ...
    so         (time, depth, latitude, longitude) float32 572kB ...
    sob        (time, latitude, longitude) float32 572kB ...
    thetao     (time, depth, latitude, longitude) float32 572kB ...
    uo         (time, depth, latitude, longitude) float32 572kB ...
    vo         (time, depth, latitude, longi

In [8]:
df_copernicus = copernicus_data.to_dataframe().reset_index()
# put column time in front
df_copernicus = df_copernicus[["time"] + [col for col in df_copernicus.columns if col != "time"]]

display(df_copernicus.info())

display(df_copernicus)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143040 entries, 0 to 143039
Data columns (total 15 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   time       143040 non-null  datetime64[ns]
 1   depth      143040 non-null  float32       
 2   latitude   143040 non-null  float32       
 3   longitude  143040 non-null  float32       
 4   bottomT    49915 non-null   float32       
 5   mlotst     49915 non-null   float32       
 6   siconc     49915 non-null   float32       
 7   sithick    49915 non-null   float32       
 8   sla        49915 non-null   float32       
 9   so         49915 non-null   float32       
 10  sob        49915 non-null   float32       
 11  thetao     49915 non-null   float32       
 12  uo         49915 non-null   float32       
 13  vo         49915 non-null   float32       
 14  wo         49915 non-null   float32       
dtypes: datetime64[ns](1), float32(14)
memory usage: 8.7 MB


None

,time,depth,latitude,longitude,bottomT,mlotst,siconc,sithick,sla,so,sob,thetao,uo,vo,wo
0,2025-01-01 00:00:00,0.501646,54.274944,10.041578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-01-01 01:00:00,0.501646,54.274944,10.041578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-01-01 02:00:00,0.501646,54.274944,10.041578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-01-01 03:00:00,0.501646,54.274944,10.041578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-01-01 04:00:00,0.501646,54.274944,10.041578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143035,2025-01-31 20:00:00,0.501646,54.524940,10.347133,4.567245,10.031350,0.0,0.0,-0.111337,17.642529,19.848560,4.294259,0.104553,-0.137242,0.000002
143036,2025-01-31 21:00:00,0.501646,54.524940,10.347133,4.565339,10.018709,0.0,0.0,-0.127637,17.571630,19.834522,4.283749,0.103176,-0.145167,0.000002
143037,2025-01-31 22:00:00,0.501646,54.524940,10.347133,4.563229,10.015547,0.0,0.0,-0.131714,17.503935,19.819601,4.271446,0.090108,-0.148308,0.000001
143038,2025-01-31 23:00:00,0.501646,54.524940,10.347133,4.560693,10.021186,0.0,0.0,-0.124443,17.430189,19.804668,4.257999,0.060021,-0.145717,0.000001


In [13]:
cols = [
    "time", "temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", 
    "precipitation_probability", "precipitation", "rain", "showers", "snowfall", "snow_depth",
    "weather_code", "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", 
    "cloud_cover_mid", "cloud_cover_high", "visibility", "evapotranspiration", 
    "et0_fao_evapotranspiration", "vapour_pressure_deficit", "wind_speed_10m", "wind_speed_80m", 
    "wind_speed_120m", "wind_speed_180m", "wind_direction_10m", "wind_direction_80m", 
    "wind_direction_120m", "wind_direction_180m", "wind_gusts_10m", "temperature_80m", 
    "temperature_120m", "temperature_180m", "soil_temperature_0cm", "soil_temperature_6cm", 
    "soil_temperature_18cm", "soil_temperature_54cm", "soil_moisture_0_to_1cm", 
    "soil_moisture_1_to_3cm", "soil_moisture_3_to_9cm", "soil_moisture_9_to_27cm", 
    "soil_moisture_27_to_81cm"]

# get unique combinations of latitude and longitude
df_lat_lon_time = df_copernicus[['latitude', 'longitude']]
df_lat_lon_time['lat_lon'] = df_lat_lon_time['latitude'].astype(str) + "," + df_lat_lon_time['longitude'].astype(str)
df_lat_lon_time = df_lat_lon_time.drop_duplicates(subset=['lat_lon'])
lat_list = df_lat_lon_time['latitude'].tolist()
lon_list = df_lat_lon_time['longitude'].tolist()

#df_lat_lon_time['time'] = df_copernicus['time'].unique()[0]
#display(df_lat_lon_time)
df_openweather_all = pd.DataFrame(columns=cols)
for idx, date in tqdm(enumerate(df_copernicus['time'].unique()), desc="OpenMeteoWeather", total=df_copernicus['time'].nunique()):
    for latitude, longitude in tqdm(zip(lat_list, lon_list),desc="Latitude & Longitude", total=len(lat_list)):

        time = date

        # bring time to isoformat
        time = time.isoformat().split('T')[0]
        
        open_meteo_weather = OpenMeteoWeather(
        latitude=[latitude],
        longitude=[longitude],
        start_date=time,
        end_date=time
        ) 
        df_openweather = open_meteo_weather.get_weather_dataframe().rename(columns={"date": "time"})
        
        # convert datetime64[ns, UTC] to datetime64[ns]
        df_openweather['time'] = df_openweather['time'].dt.tz_localize(None)

        #df_openweather.dropna(axis=0, how='any', inplace=True)
        df_openweather['latitude'] = latitude
        df_openweather['longitude'] = longitude
        
        df_merged = pd.merge(df_copernicus, df_openweather, on=['time', 'latitude', 'longitude'], how='inner')
        display(df_merged.info())
        break
    break
        #df_openweather_all = pd.concat([df_openweather_all, df_openweather], ignore_index=True)
        
        


/var/folders/zf/vdfq8dnn2yj6j643_88nnsk80000gn/T/ipykernel_4241/3298097271.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lat_lon_time['lat_lon'] = df_lat_lon_time['latitude'].astype(str) + "," + df_lat_lon_time['longitude'].astype(str)
OpenMeteoWeather:   0%|          | 0/745 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 57 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   time                        23 non-null     datetime64[ns]
 1   depth                       23 non-null     float32       
 2   latitude                    23 non-null     float32       
 3   longitude                   23 non-null     float32       
 4   bottomT                     0 non-null      float32       
 5   mlotst                      0 non-null      float32       
 6   siconc                      0 non-null      float32       
 7   sithick                     0 non-null      float32       
 8   sla                         0 non-null      float32       
 9   so                          0 non-null      float32       
 10  sob                         0 non-null      float32       
 11  thetao                      0 non-null      float32       
 

None

OpenMeteoWeather:   0%|          | 0/745 [00:00<?, ?it/s]


In [ ]:
df_openweather_all

,time,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation_probability,precipitation,rain,showers,snowfall,...,temperature_180m,soil_temperature_0cm,soil_temperature_6cm,soil_temperature_18cm,soil_temperature_54cm,soil_moisture_0_to_1cm,soil_moisture_1_to_3cm,soil_moisture_3_to_9cm,soil_moisture_9_to_27cm,soil_moisture_27_to_81cm


In [ ]:
df_copernicus['time'].unique().tolist()

[]

In [ ]:
# bring latitude and longitude to the front
df_openweather_all = df_openweather_all[["time", "latitude", "longitude"] + [col for col in df_openweather_all.columns if col not in ["time", "latitude", "longitude"]]]
df_openweather_all.dropna(axis=1, how='any', inplace=True)
df_openweather_all['time'] = pd.to_datetime(df_openweather_all['time'], utc=False).dt.tz_localize(None)
display(df_openweather_all.head(3))
df_openweather_all.info()

KeyError: "['latitude', 'longitude'] not in index"

In [ ]:
df_merged = pd.merge(df_copernicus, df_openweather_all, on=["time", "latitude", "longitude"], how="inner")
df_merged

,time,latitude,longitude,bottomT,mlotst,siconc,sithick,sla,so,sob,...,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_direction_10m,wind_gusts_10m
0,2024-01-01 00:00:00,55.491592,9.597136,4.26668,3.181917,0.0,0.0,0.109804,17.312731,17.313353,...,997.799988,95.0,5.0,28.0,93.0,0.002023,0.080862,18.089775,174.289505,29.879999
1,2024-01-01 00:00:00,55.491592,9.597136,4.26668,3.181917,0.0,0.0,0.109804,17.312731,17.313353,...,997.799988,95.0,5.0,28.0,93.0,0.002023,0.080862,18.089775,174.289505,29.879999
2,2024-01-01 00:00:00,55.491592,9.597136,4.26668,3.181917,0.0,0.0,0.109804,17.312731,17.313353,...,997.799988,95.0,5.0,28.0,93.0,0.002023,0.080862,18.089775,174.289505,29.879999
3,2024-01-01 00:00:00,55.491592,9.597136,4.26668,3.181917,0.0,0.0,0.109804,17.312731,17.313353,...,997.799988,95.0,5.0,28.0,93.0,0.002023,0.080862,18.089775,174.289505,29.879999
4,2024-01-01 00:00:00,55.491592,9.597136,4.26668,3.181917,0.0,0.0,0.109804,17.312731,17.313353,...,997.799988,95.0,5.0,28.0,93.0,0.002023,0.080862,18.089775,174.289505,29.879999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148147,2024-01-01 22:00:00,55.608257,11.124907,5.09846,10.007136,0.0,0.0,-0.110556,18.155640,20.309689,...,1001.799988,100.0,100.0,43.0,0.0,0.000000,0.029744,9.686609,138.012863,22.680000
148148,2024-01-01 22:00:00,55.608257,11.124907,5.09846,10.007136,0.0,0.0,-0.110556,18.155640,20.309689,...,1001.799988,100.0,100.0,43.0,0.0,0.000000,0.029744,9.686609,138.012863,22.680000
148149,2024-01-01 22:00:00,55.608257,11.124907,5.09846,10.007136,0.0,0.0,-0.110556,18.155640,20.309689,...,1001.799988,100.0,100.0,43.0,0.0,0.000000,0.029744,9.686609,138.012863,22.680000
148150,2024-01-01 22:00:00,55.608257,11.124907,5.09846,10.007136,0.0,0.0,-0.110556,18.155640,20.309689,...,1001.799988,100.0,100.0,43.0,0.0,0.000000,0.029744,9.686609,138.012863,22.680000


In [ ]:
OpenMeteoWeather = OpenMeteoWeather(
    latitude=55.5,
    longitude=10.5,
    start_datetime=START_DATE,
    end_datetime=END_DATE,

) 

array([55.491592, 55.50826 , 55.524925, 55.54159 , 55.558258, 55.574924,
       55.59159 , 55.608257], dtype=float32)

In [ ]:
# ------------ Bring data to the frontend ------------
app = fastapi.FastAPI()